In [5]:
# from transformers import AutoTokenizer, EsmForSequenceClassification
import torch
import pandas as pd
import esm

In [4]:
# set(pd.read_csv("../transformer/pfam_tiny_1579.train.csv")['label'])

{'PF01250.17', 'PF02261.16', 'PF03015.19', 'PF06094.12'}

In [7]:
def torch_mem_info_reset():
    device = 'cuda:0'
    alloc = torch.cuda.memory_allocated(device) / (1024*1024)
    res = torch.cuda.memory_reserved(device) / (1024*1024)
    max_alloc = torch.cuda.max_memory_allocated(device) / (1024*1024)
    max_res = torch.cuda.max_memory_reserved(device) / (1024*1024)
    # torch.cuda.empty_cache()
    # torch.cuda.reset_peak_memory_stats()
    print(f"Allocated: {alloc} Reserved: {res} Max_alloc: {max_alloc} Max_res: {max_res}")


def count_model_size_mb(model):
    return sum(p.nelement() * p.element_size() for p in model.parameters()) / (1024 * 1024)

def count_parameters(model):
    return sum(p.nelement() for p in model.parameters())

In [108]:
torch.cuda.reset_peak_memory_stats()
torch_mem_info_reset()

Allocated: 142.88037109375 Reserved: 6282.0 Max_alloc: 142.88037109375 Max_res: 6282.0


In [6]:
# Can be used
# model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm2_t6_8M_UR50D")
# model, alphabet = esm.pretrained.esm2_t12_35M_UR50D()
model, alphabet = esm.pretrained.esm2_t30_150M_UR50D()

In [15]:
count_model_size_mb(model)
next(model.parameters())

torch.Size([33, 640])

In [7]:
# model has a base class nn.Module
print(esm.model.esm2.ESM2.__bases__)
# model

(<class 'torch.nn.modules.module.Module'>,)


In [102]:
from torch.nn import Identity
model.contact_head = Identity()
model.emb_layer_norm_after = Identity()
model.lm_head = Identity()
model

In [12]:
df = pd.read_csv("../transformer/pfam_tiny_1579.train.csv")
df

,original,corrupted,label
0,M L R M M M N S K I H R A T V T E A D L N Y V ...,M L R M M M N S K I H R A T V T E A D L N Y V ...,PF02261.16
1,L F V Y G T L R Q G E S N H N F L A D S Q C L ...,L F V Y G T L R Q G E S N H N F L A D S Q C L ...,PF06094.12
2,P Y E L M V L L R P D L A D D R L E A A L E R ...,P Y E L M V L L R P D L X D D R L V A A L E R ...,PF01250.17
3,Y E L F A L A K P Q L G K K A I A S I M K A V ...,Y E L F A L A K P V L G T K A I A S I M K A V ...,PF01250.17
4,Q I K L M H A K L H H L R V T Q A E L D Y V G ...,Q I K L M H A K L H H L R V T Q A E L D Y V G ...,PF02261.16
...,...,...,...
1574,F Q K L P A Y I I D F G A W I T G R K P M Q V ...,F Q K V P A Y I I D F G A W I T G E K P M Q V ...,PF03015.19
1575,L Y E T M Y I I R P D L G E E T V D Q V I N Q ...,L Y E T M Y I I R P D L G E E T V D Q V I N Q ...,PF01250.17
1576,K Y E V M Y I V K P T L D E E A R T A V I A K ...,K Y N V M Y I V K P T L D E E A R T A V I A K ...,PF01250.17
1577,A Y E I L Y I I R P D M D E E A T N A L V D R ...,A Y E I L Y I I R P D M D E E A T N A L V D R ...,PF01250.17


In [13]:
seq_test = df['original'][2] * 20

In [14]:
len(seq_test)

3700

In [17]:
# len(seq_test)*100*480*4 / (1024*1024)

In [18]:
# free / (1024*1024)
def check_gpu_mem(device):
    free, total = torch.cuda.mem_get_info(device)
    print(f"Free: {free / (1024*1024)} Total: {total / (1024*1024)}")

def get_tensor_size_mb(tens):
    sz = tens.element_size() * tens.nelement()
    sz /= (1024 * 1024)
    print(f"Size MiB: {sz}")


In [105]:
batch_converter = alphabet.get_batch_converter()
data = [
    # ("protein1", seq_test),
    ("protein2", seq_test) for _ in range(20)
    # ("prot1", "ADPALVLH"),
    # ("prot2", "ADPAL"),
    # ("prot3", "AD")
]
batch_labels, batch_strs, batch_tokens = batch_converter(data)

In [27]:
device = torch.device("cuda:0")
# alphabet.tok_to_idx
# '<cls>': 0,
#  '<pad>': 1,
#  '<eos>': 2,
#  '<unk>': 3,
# ...
# '.': 29,
#  '-': 30,
#  '<null_1>': 31,
#  '<mask>': 32}

In [93]:
# torch.cuda.is_available()
print(batch_tokens.device)
# torch.cuda.device_count()
print(batch_tokens[0].dtype)
check_gpu_mem(device)
batch_tokens_gpu64 = batch_tokens.to(device)
batch_tokens_gpu8 = batch_tokens.to(device).to(dtype=torch.uint8)
# batch_tokens = batch_tokens.to(torch.int16)
get_tensor_size_mb(batch_tokens)
get_tensor_size_mb(batch_tokens_gpu64)
get_tensor_size_mb(batch_tokens_gpu8)
batch_tokens_gpu8.dtype

cpu
torch.int64
Free: 21211.0 Total: 22515.75
Size MiB: 0.142059326171875
Size MiB: 0.142059326171875
Size MiB: 0.017757415771484375


torch.uint8

In [94]:
# torch.cuda.empty_cache()
# check_gpu_mem(device)
# get_tensor_size_mb(batch_tokens)
# batch_tokens.shape
check_gpu_mem(device)
get_tensor_size_mb(batch_tokens)

Free: 21211.0 Total: 22515.75
Size MiB: 0.142059326171875


In [95]:
# tot = torch.cuda.memory_reserved(device)
# free, total = torch.cuda.mem_get_info(device)
# tot / (1024*1024)
batch_tokens.requires_grad

False

In [19]:
import gc 
del batch_tokens
gc.collect()
torch.cuda.empty_cache()

In [69]:
print("Number of parameters: ", count_parameters(model))
print("Model size [MB]: ", count_model_size_mb(model))
# para = next(model.parameters())
# para.nelement()

Number of parameters:  33501394
Model size [MB]:  127.79767608642578


In [71]:
# next(model.parameters()).dtype
# 35*1e6*4/(1024*1024*1024)
model.to(torch.bfloat16)
next(model.parameters())

Parameter containing:
tensor([[ 0.0525, -0.2617, -0.0292,  ..., -0.0635, -0.0179,  0.0432],
        [-0.1777, -0.5977,  0.1943,  ...,  0.3672,  0.1357,  0.0131],
        [ 0.0981, -0.2754,  0.0186,  ..., -0.0469,  0.1670, -0.0327],
        ...,
        [ 0.0022, -0.0425,  0.0080,  ...,  0.0479, -0.0464, -0.0066],
        [-0.0014, -0.0260,  0.0347,  ...,  0.0342, -0.0659, -0.0422],
        [-0.1807, -0.5977,  0.1934,  ...,  0.3711,  0.1455,  0.0138]],
       dtype=torch.bfloat16, requires_grad=True)

In [97]:
count_parameters(model)

33501394

In [98]:
print(next(model.parameters()).is_cuda)
# model.cuda()
print(next(model.parameters()).is_cuda)


True
True


In [77]:
check_gpu_mem(device)
print(next(model.parameters()).is_cuda)

Free: 22061.0 Total: 22515.75
True


In [99]:
batch_tokens_gpu8

tensor([[ 0, 14, 19,  ...,  7, 10,  2],
        [ 0, 14, 19,  ...,  7, 10,  2],
        [ 0, 14, 19,  ...,  7, 10,  2],
        ...,
        [ 0, 14, 19,  ...,  7, 10,  2],
        [ 0, 14, 19,  ...,  7, 10,  2],
        [ 0, 14, 19,  ...,  7, 10,  2]], device='cuda:0', dtype=torch.uint8)

In [101]:
# model = model.to(device)
model.eval()
with torch.no_grad():
    res = model(batch_tokens_gpu64)

In [110]:
torch.cuda.reset_max_memory_cached()
torch.cuda.reset_max_memory_allocated()
torch.cuda.reset_peak_memory_stats()

/home/nikola_dev/miniconda3/envs/ssi/lib/python3.10/site-packages/torch/cuda/memory.py:329: FutureWarning: torch.cuda.reset_max_memory_cached now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(
/home/nikola_dev/miniconda3/envs/ssi/lib/python3.10/site-packages/torch/cuda/memory.py:303: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


In [111]:
torch_mem_info_reset()
# check_gpu_mem(device)

Allocated: 142.87841796875 Reserved: 6282.0 Max_alloc: 142.87841796875 Max_res: 6282.0


In [89]:
get_tensor_size_mb(res['logits'])

Size MiB: 0.23439788818359375


In [21]:
res

{'logits': tensor([[[ 2.5057,  6.8665,  1.0299,  ...,  8.5440, -1.4410, -7.0620],
          [ 4.4105,  2.9949, -1.7162,  ...,  4.4722, -0.3008, -2.9338],
          [-0.7679, -2.4428, -1.0123,  ..., -1.6110,  2.6296,  0.2919],
          ...,
          [-1.7249, -1.8000,  4.2837,  ...,  1.0689, -0.8234, -5.0085],
          [-0.1915, -0.7245,  2.4475,  ..., -0.7194,  0.2490, -4.1189],
          [-1.1293,  1.0655,  1.3112,  ...,  1.8803, -6.6340, -4.0444]],
 
         [[ 2.5057,  6.8665,  1.0299,  ...,  8.5440, -1.4410, -7.0620],
          [ 4.4105,  2.9949, -1.7162,  ...,  4.4722, -0.3008, -2.9338],
          [-0.7679, -2.4428, -1.0123,  ..., -1.6110,  2.6296,  0.2919],
          ...,
          [-1.7249, -1.8000,  4.2837,  ...,  1.0689, -0.8234, -5.0085],
          [-0.1915, -0.7245,  2.4475,  ..., -0.7194,  0.2490, -4.1189],
          [-1.1293,  1.0655,  1.3112,  ...,  1.8803, -6.6340, -4.0444]]]),
 'representations': {6: tensor([[[ 2.5057,  6.8665,  1.0299,  ...,  8.5440, -1.4410, -7.06